# TinyML Training → TFLite INT8 → Accuracy
Fast training + quantization; reuse accuracy in the simulator.

In [ ]:

import tensorflow as tf, numpy as np, os
tf.random.set_seed(123)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.astype("float32")/255.)[..., None]
x_test  = (x_test.astype("float32")/255.)[...,  None]

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, 3, activation="relu", input_shape=(28,28,1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, 3, activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy", metrics=["accuracy"])
_ = model.fit(x_train, y_train, epochs=3, batch_size=256, validation_split=0.1, verbose=0)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"FP32 Keras accuracy: {test_acc:.4f}")


In [ ]:

def representative_data_gen():
    for i in range(100):
        idx = np.random.randint(0, len(x_train))
        sample = x_train[idx:idx+1]
        yield [sample.astype(np.float32)]

# Full int8 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type  = tf.int8
converter.inference_output_type = tf.int8
tflite_int8 = converter.convert()

os.makedirs("models", exist_ok=True)
open("models/mnist_cnn_int8.tflite", "wb").write(tflite_int8)
print("Saved: models/mnist_cnn_int8.tflite")


In [ ]:

def run_tflite_int8(path, max_samples=2000):
    interpreter = tf.lite.Interpreter(model_path=path)
    interpreter.allocate_tensors()
    in_details  = interpreter.get_input_details()
    out_details = interpreter.get_output_details()
    scale, zero = in_details[0]["quantization"]
    correct = 0
    N = min(max_samples, len(x_test))
    for i in range(N):
        x = x_test[i:i+1]
        xq = (x/scale + zero).round().astype(np.int8)
        interpreter.set_tensor(in_details[0]['index'], xq)
        interpreter.invoke()
        logits = interpreter.get_tensor(out_details[0]['index'])
        pred = np.argmax(logits, axis=1)[0]
        correct += (pred == y_test[i])
    return correct / N

acc_int8 = run_tflite_int8("models/mnist_cnn_int8.tflite", max_samples=2000)
print(f"TFLite INT8 accuracy: {acc_int8:.4f}")


**Copy** the printed `TFLite INT8 accuracy` and paste it in the simulator notebook (cell with `set_int8_accuracy(...)`).